# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

Esses valores podem ser atribuídos na função `main`, não é necessária leitura pelo teclado. Entretanto, o programa deve ser genérico, ou seja, funcionará para qualquer outro valor atribuído às 3 variáveis.

In [ ]:
#include<stdio.h>

int main(){
    
    int n;
    float s;
    float j;
    
    s=200;
    n=5;
    j=1;
    
    float f = s;
    
   for(int i=0; i<n; i++){
       printf("%3.2f\n", f);
       f = f + (f * j/100);
   }
    
    return 0;
}

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de cada parcela X do empréstimo.

Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa em que está inserida. Considere que esta função poderia ser eventualmente reusada em outro programa e deveria ser possível fazê-lo sem que ela dependa de nada do programa em que irá se inserir.

A função deve apenas calcular uma única parcela em cada chamada, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [ ]:
#include<stdio.h>

//Recebe os dados para cálculo das parcelas e retorna o valor da parcela X
float proximaParcela(float s, int n, float j, float *parcela, int *X){
    if (*X == 0){
        *parcela = s;
    }
    else if (*X <= n){
        *parcela = *parcela + (*parcela * j/100);
    }
    else{
        *parcela = 0;
    }
    (*X)++;
    return *parcela;
}

int main(){
    int n;
    int X;
    float s;
    float j;
    float f;
   
    s=200;
    n=5;
    j=1;
    X=0;
    f=s;
    
    for(int i=0; i<n; i++){
        f = proximaParcela(s, n, j, &f, &X);
        printf("O preço da parcela %d será: %3.2f\n", X, f);
    }
    
    return 0;
}

## Exercício Parte 3 - Minimizando os Parâmetros

Na solução anterior, você deve ter usado vários parâmetros para a comunicação entre o `main` com a função `proximaParcela`, evitando usar variáveis globais. Modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar todos os valores que estejam relacionados a um empréstimo.

In [ ]:
//Para minimiza, faremos uma struct com os dados do empréstimo
#include <stdio.h>

typedef struct {
    int n;
    float s;
    float j;
    int X;
    float parcela;
}Emprestimo;

void newEmprestimo(Emprestimo *emprestimo_1){
    emprestimo_1 -> X = 1;
    emprestimo_1 -> parcela = emprestimo_1 -> s;
}

float proximaParcela(Emprestimo *emprestimo){
    float ret = emprestimo -> parcela;
    
    if(emprestimo->X < emprestimo->n){
        emprestimo->parcela = emprestimo->parcela + (emprestimo->parcela * (emprestimo->j/100));
    }
    else
        emprestimo->parcela = 0;
    
    (emprestimo->X)++;
    
    return ret;
}
    
int main(){
    Emprestimo emprestimo_1;
    
    emprestimo_1.s = 200;
    emprestimo_1.n = 5;
    emprestimo_1.j = 1;
    
    newEmprestimo(&emprestimo_1);
    
    int i =1;
    float f;
    do{
        f = proximaParcela(&emprestimo_1);
        printf("O valor da parcela %d eh %3.2f\n", i, f);
        i++;
    }while(f>0);
    
    return 0;
}

## Exercício Parte 4 - Mantendo o Estado

A função que você codificou na solução anterior provavelmente não é capaz de manter o controle do que chamaremos de **estado do empréstimo**, ou seja, dados de qual a parcela corrente ou da última parcela calculada.

Modifique a função (módulo) `proximaParcela` acima de modo que ela seja capaz de guardar o **estado do empréstimo** por sua conta, sem que o `main` precise ficar informando este estado por parâmetros (por exemplo). Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência das funções com o programa em que ela está inserida.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente. O ideal é que o main não tenha acesso ao modo como as funções vão representar o **estado do empréstimo**, para que toda a lógica do empréstimo fique dentro das funções.

In [3]:
#include <stdio.h>

int n;
float s;
float j;
int X;
float parcela;

void newEmprestimo(float s_,int n_, float j_){
    n = n_;
    s = s_;
    j = j_;
    X = 1;
    parcela = s;
}

float proximaParcela(){
    float ret = parcela;
    X++;
    if(X <= n){
        parcela = parcela + (parcela * j/100); 
    }
    else
        parcela = 0;
    
    return ret;
}

int main(){
    
    newEmprestimo(200,5,1);
    
    int i =1;
    float f;
    do{
        f = proximaParcela();
        printf("O valor da parcela %d eh %3.2f\n", i, f);
        i++;
    }while(f>0);
    
    return 0;
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12
O valor da parcela 6 eh 0.00


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada **estado de empréstimo** separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, as funções de cálculo do empréstimo ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que comporte até 100 empréstimo e, cada novo empréstimo, só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo (até o limite de 100).

O seu programa deve apresentar o exemplo acima como teste.

In [7]:
//Para minimiza, faremos uma struct com os dados do empréstimo
#include <stdio.h>

typedef struct {
    int n;
    float s;
    float j;
    int X;
    float parcela;
}Emprestimo;

Emprestimo newEmprestimo(float s, int n, float j){
    Emprestimo emprestimo_x;
    emprestimo_x.s = s;
    emprestimo_x.n = n;
    emprestimo_x.j = j;
    emprestimo_x.X = 1;
    emprestimo_x.parcela = s;
    return emprestimo_x;
}

float proximaParcela(Emprestimo *emprestimo){
    float ret = emprestimo -> parcela;
    
    if(emprestimo->X < emprestimo->n){
        emprestimo->parcela = emprestimo->parcela + (emprestimo->parcela * (emprestimo->j/100));
    }
    else
        emprestimo->parcela = 0;
    
    (emprestimo->X)++;
    
    return ret;
}
    
int main(){
    Emprestimo emprestimo_1;
    Emprestimo emprestimo_2;
    
    emprestimo_1 = newEmprestimo(200,5,1);
    emprestimo_2 = newEmprestimo(500,7,2);
    
    int i =1;
    float f_1;
    float f_2;
    
    do{
        f_1 = proximaParcela(&emprestimo_1);
        printf("O valor da parcela %d do Empréstimo 1 é %3.2f\n", i, f_1);
        f_2 = proximaParcela(&emprestimo_2);
        printf("O valor da parcela %d do Empréstimo 2 é %3.2f\n", i, f_2);
        i++;
    }while(f_1>0 || f_2>0);
    
    return 0;
}

O valor da parcela 1 do Empréstimo 1 é 200.00
O valor da parcela 1 do Empréstimo 2 é 500.00
O valor da parcela 2 do Empréstimo 1 é 202.00
O valor da parcela 2 do Empréstimo 2 é 510.00
O valor da parcela 3 do Empréstimo 1 é 204.02
O valor da parcela 3 do Empréstimo 2 é 520.20
O valor da parcela 4 do Empréstimo 1 é 206.06
O valor da parcela 4 do Empréstimo 2 é 530.60
O valor da parcela 5 do Empréstimo 1 é 208.12
O valor da parcela 5 do Empréstimo 2 é 541.22
O valor da parcela 6 do Empréstimo 1 é 0.00
O valor da parcela 6 do Empréstimo 2 é 552.04
O valor da parcela 7 do Empréstimo 1 é 0.00
O valor da parcela 7 do Empréstimo 2 é 563.08
O valor da parcela 8 do Empréstimo 1 é 0.00
O valor da parcela 8 do Empréstimo 2 é 0.00
